In [2]:
#Alana Laryssa, Cauanne Linhares, Geysa Fernandes
#Brazilian Scientific Mobility Program 2015-2016

#This code is meant to generate an artificial problem for the lakewater.
#It basically generate .txt files for the bacterias database, virus and also for the lake water.
#You may change the bacteria, virus or corruption rate on the specific variables, to generate different dat as it suits you

#Libraries imports, some of these libraries need to be installed prior to use
#We are using the Jupyter Notebook, that may help if some of the libraries does not work
from random import choice
from string import ascii_uppercase
import math
import time
from swalign import swalign #You need to have the swallign file in the same folder as the code file
from scipy.stats import beta
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps
from numpy import trapz
import random
import operator

print(swalign)
# ****** functions ********
#returns a random string of specified length
#length: strign length
def randomword(length):
    return (''.join(choice('ACGT') for i in range(0, length)))

#retuns an array of random strings
#size: how many strings there will be in the array
#lakeMinLen: min sequence length
#lakeMaxLen: max sequence length
def lakeString(size, lakeMinLen, lakeMaxLen):     
    lake_water = []
    for i in range(0, size):
        random.seed()
        #generates a random sequence length
        y = random.randint(lakeMinLen, lakeMaxLen)
        
        _str = randomword(y)
        lake_water.append(_str)
    return lake_water

#returns one array of strings that represents database data set
#dbMinSeqLen: min string length
#dbMaxSeqLen: max string length
#dbSize: max number sequences in the array
def dbStrings(dbMinSeqLen, dbMaxSeqLen, dbSize):
    _array = []
    #generates how many different sequences there will be for the array
    random.seed()
    #x = random.randint(2, dbSize)
    x = 30

    for i in range(0, x):
        #generates a random sequence size
        random.seed()
        y = random.randint(dbMinSeqLen, dbMaxSeqLen)
        _str = randomword(y)
        _array.append(_str)
    return _array

#corrupts the string if it is under the corruption rate
def stringCorruption(strg, corrupt_rate):
    _str = ''
    for k in strg:
        #get a random number between 0 and 1
        random.seed()
        floatr = random.random()
        if  floatr > corrupt_rate :
            _str = _str + k
        else:
            _str = _str + choice('ACGT')     
    return _str

#configures the controlled data
#parameters:
#k_v: known_virus
#k_b: known_bacterias
#nControlled: how many records will be controlled
#error: corruption limit
def dataController(k_v, k_b, error, nControlVir, nControlBact, lakeMinLen, lakeMaxLen):
    controlled_lake = []
    vLen = len(known_viruses)
    bLen = len(known_bacterias)
    
    #creates the specified amount of controlled lake sequences with viral pieces
    for j in range (0, nControlVir):
        #get a random virus
        vir = known_viruses[random.randint(1, vLen-1)]
        #get the length of the random virus
        virLen = len(vir)            
        count = 0
         #tries to find valid random substrings to be used
        num1 = random.randint(10, virLen)
        num2 = num1 - random.randint(1, num1)    
        print("N1:", num1, "N2:", num2)

        #get a substring
        strg = vir[num2:num1]
        controlled_lake.append(strg)  
    
    #creates the specified amount of controlled lake sequences with bacterial pieces
    for j in range (0, nControlBact):
        bact = known_bacterias[random.randint(1, bLen-1)]
        bactLen = len(bact)            
        count = 0
        #tries to find valid random substrings to be used
        num1 = random.randint(10, bactLen)
        num2 = num1 - random.randint(1, num1)    
        print("N1:", num1, "N2:", num2)

        #get a substring
        strg = bact[num2:num1]
        controlled_lake.append(strg)  
    
    return controlled_lake
    
# ******************************************************* main ****************************************************

####---------------- config. variables --------------------------------####
percentage = 1      #how many will have virus/bacteria pieces
#Change the bacterialRate variable for what suits your problem
bacterialRate = 0.4 #how many of the controlled lake sequences will bacterial piece
error = 0.0         #corruption rate
distLimit = 0       #distance between matching sequences

lakeSize = 50    #max number of lake sequences
lakeMinLen = 10     #min lake string length
lakeMaxLen = 250     #max lake string length

dbSize = 550  #max number sequences in the array
dbMaxSeqLen = 500  #max db string length
dbMinSeqLen = lakeMinLen #min db string length

#print configuration
print("\n\nConfig. Variables")
print("  Controlled Data:", percentage*100, "%")
print("  Percentage of Bact. data:", bacterialRate*100, "%")
print("  Corruption Rate:", error*100, "%")
print("  Ditance Limit:", distLimit)
print("  Db string min lenght:", dbMinSeqLen)
print("  Db string max lenght:", dbMaxSeqLen)
print("  Db max size:", dbSize)
print("  Lake string min lenght:", lakeMinLen)
print("  Lake string max lenght:", lakeMaxLen)
print("  Lake data max size:", lakeSize, "\n\n")

#generates how many different sequences there will be for the lake
random.seed()
size = random.randint(2, lakeSize)

#Variable that holds the number of samples from the lake
#size = 10

#generates the database representation
known_viruses = dbStrings(dbMinSeqLen, dbMaxSeqLen, dbSize)
known_bacterias = dbStrings(dbMinSeqLen, dbMaxSeqLen, dbSize)

#generates the lake sequences w/ the specified percentage of controlled data
P = math.ceil(size * percentage)
lake = lakeString(size - P, lakeMinLen, lakeMaxLen) #produce a part with totally random sequences  

#calculates how many sequences will be viral and how many will be bacterial
nControlBact = int(math.ceil(P * bacterialRate))
if bacterialRate == 0.0:
    nControlVir = P
else:
    nControlVir = int((nControlBact * (1 - bacterialRate)) / bacterialRate)

print("# inserted viral pieces:", nControlVir, "\n# inserted bacterial pieces: ", nControlBact)
size = size - P + nControlVir + nControlBact
print("True bacterial percentage:", nControlBact/size*100, "%")

#produce the other part with viral or bacterial pieces
controlled = dataController(known_viruses, known_bacterias, error, nControlVir, nControlBact, lakeMinLen, lakeMaxLen)
lake = lake + controlled #join the sequences

#Lakewater String
#For 100% controlled
fd = open("../database/lake_data.txt", "w+")
for i in lake:
    fd.write(i)
    fd.write('\n')
fd.close()

#Virus String
fd_v = open("../database/virus_db.txt", "w+")
for i in known_viruses:
    fd_v.write(i)
    fd_v.write("\n")
fd_v.close()

#Bacteria String
fd_b = open("../database/bact_db.txt", "w+")
for i in known_bacterias:
    fd_b.write(i)
    fd_b.write("\n")
fd_b.close()

print("end of execution...")

<module 'swalign.swalign' from 'C:\\Users\\Geysa\\Documents\\GitRepo\\codes\\swalign\\swalign\\__init__.py'>


Config. Variables
  Controlled Data: 100 %
  Percentage of Bact. data: 40.0 %
  Corruption Rate: 0.0 %
  Ditance Limit: 0
  Db string min lenght: 10
  Db string max lenght: 500
  Db max size: 550
  Lake string min lenght: 10
  Lake string max lenght: 250
  Lake data max size: 50 


# inserted viral pieces: 25 
# inserted bacterial pieces:  17
True bacterial percentage: 40.476190476190474 %
N1: 86 N2: 21
N1: 52 N2: 7
N1: 85 N2: 4
N1: 290 N2: 138
N1: 64 N2: 9
N1: 10 N2: 4
N1: 85 N2: 32
N1: 102 N2: 62
N1: 245 N2: 151
N1: 25 N2: 13
N1: 60 N2: 4
N1: 186 N2: 134
N1: 179 N2: 130
N1: 129 N2: 102
N1: 181 N2: 116
N1: 481 N2: 293
N1: 92 N2: 77
N1: 308 N2: 126
N1: 274 N2: 121
N1: 437 N2: 434
N1: 367 N2: 265
N1: 187 N2: 166
N1: 327 N2: 166
N1: 43 N2: 26
N1: 74 N2: 5
N1: 108 N2: 60
N1: 286 N2: 210
N1: 21 N2: 7
N1: 69 N2: 53
N1: 40 N2: 14
N1: 11 N2: 5
N1: 21 N2: 3
N1: 20 N2: 11
N1: 55 N2: 45